# Platform Positioning

In [1]:
from __future__ import print_function

In [41]:
import random as random #For random initial platform position
import numpy as np
import pandas as pd

import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point, LineString 
import shapely
from geopy import distance # For calculating distance between two points 


import pickle  as pkl # For importing pickled objects

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from math import tan, isclose, pi #F or calculating point on circumference - step out radius

In [42]:
import os
os.getcwd()

'/Users/Pri.balachandran@ibm.com/Desktop/BP/20191106_PlatformWCAnalysis'

# Data Ingestion & Cleaning
Since Well Concept data is created based on the cleaned Target data, the notebook has been structured with Targets, then Well Concepts

##  Target Geometry Data
This has been queried and pickled. The column names refer to the SQL function performed on geometry.
e.g. geom.AsTextZM() AS geom_text_zm

## Well Concept Data
As no suitable test data exists, dummy data is generated. There is a separate dummdatacreator.py file containing this work.

Rather than have Well Concepts randomly placed anywhere, we restrict it to only be within Target Polygon space

In [17]:
l = []



In [18]:
from dummydatacreator import  GenerateWellConcept # This is not supported in Azure notebooks

In [22]:
WC_name, WC_points, WC_resource = UpdateWellConceptData(gdf_targPoly)

print("Well Concepts Generated:\n",'\n'.join('Name: {}, Points: {}, Resource: {}'.format(WC_name[k], WC_points[k], WC_resource[k]) for k in range(5)))

Well Concepts Generated:
 Name: V-0, Points: [-60.343110061108035, 10.1188300528309], Resource: 1.9776760230385684
Name: G-1, Points: [-60.365687872984445, 10.050755818758784], Resource: 8.754875342665866
Name: Eb-2, Points: [-60.30885129003035, 10.115099173085023], Resource: 96.33729714258996
Name: K-22, Points: [-60.303576064229276, 10.104840324652479], Resource: 17.87937471189474
Name: C-22, Points: [-60.30746941615207, 10.118283159060304], Resource: 114.76743900494468


# Mapping
In order to construct the map containing Targets, Well Concepts, Platforms, some functions are required:


### Platform Step-Out Radius Functions
As there is no method for deriving the circumference around a point in EPSG:4326, functions are required to calculate this.
- A function to find a single point on the circumference (within a given tolerance)
- A function to find multiple points all around the circumference, sufficient for plotting the circle

In [40]:
def GetXYatCircumference(centre, desired_radius, theta, tolerance = 0.00001):
    '''Calculates the X,Y coordinates of a point on the circumference of a circle, 
    with a given desired radius and angle to the vertical.
    
    Theta in radians
    '''
    
    #Initialise x,y coordinates
    x = centre[0]
    y = centre[1]

    
    #Set initial increment values
        #If we are working in the lower two quadrants, i.e. where y will be less than centre y
    if theta > pi/2 and theta < (3*pi)/2:
        y_increment=-0.01
    else:
        y_increment=0.01

        
    #Loop until the radius of the point and centre, is close to the desired radius
    while isclose(distance.distance(centre, [x,y]).kilometers,  desired_radius ,rel_tol=0.00001) == False:

        # Add the increment whilst the distance is less than desired
        while distance.distance(centre, [x,y]) < desired_radius:
            x_increment = y_increment * tan(theta)
            x += x_increment
            y += y_increment

        # Subtract the increment whilst the distance is more than desired
        while distance.distance(centre, [x,y]) > desired_radius:
            x_increment = y_increment * tan(theta)
            x -= x_increment
            y -= y_increment

        #Reduce the size of the increment to increase precision  
        y_increment = y_increment/4

    return [x,y]

In [41]:
def GetCircumferenceXY(centre, radius, n_points, tolerance = 0.00001):
    '''Calculates x,y coordinates of a specified number points on the circumference of a 
    circle with a specified centre    
    
    NB: n_points must odd, to ensure we do not get a trig erro
    '''
    assert n_points % 2 != 0, "n_points must not be an even number - gives a trig error"

    # The angle between each radius        
    theta_increment = (2*pi)/n_points

    # The angles of all the radii we will use
    thetas = [theta_increment * i for i in range(n_points)]
    
    # Get a list of all calculated XYs on the circumference
    xys = [GetXYatCircumference(centre, radius,theta_i,tolerance) for theta_i in  thetas]
    
    # Create a full 'loop' so the first and last points are the same - Polygon
    xys.append(xys[0])
    
    
    xs = [xy[0] for xy in xys]
    ys = [xy[1] for xy in xys]
    
    return xs, ys

GetCircumferenceXY([0,0], 2, 3)

([0.0, 0.015637707705740515, -0.015637707705740488, 0.0],
 [0.01796615600585938,
  -0.009028434753417969,
  -0.009028434753417969,
  0.01796615600585938])

### Well Concept Relative Position
i.e. Are Well Concepts within specified radii around a point?

In [42]:
def CalculateRankOfWCs(WC, plat, radii):
    '''Well Concepts are ranked according to binned distance from platform. i.e. 
    Given input radii [0,2,5], the function will return a list of rankings where:
    0 indicates Well Concept is more than 5 km from platform
    1 indictates Well Concept is between 2 and 5 km from platform... etc
    '''
    #Radii must be ordered, i.e. [0,2,5]
    radii = np.sort(radii)
    
    num_radii = len(radii)

    #calculate distances of all well concepts from platform 
    distances = [distance.distance(wc_i, plat) for wc_i in WC] #Karney 2013 distance
    
    distances = np.reshape(distances,[-1])

    
    #Reverse ranking - where 0 is closest to platform
    revranks = np.digitize(distances, radii, right = False) #i.e. includes left bound = 0

    # Ranking - where 0 is furthest from platform
    ranking = [num_radii - revrank for revrank in revranks]
    
    return ranking

In [43]:
dct_colors = {0: "#808080",
          1:"#FF8000",
          2:"#00CC00"}

In [44]:
def GetWCColors(WC_Points, plat_x, plat_y,radii):
    '''
    Given the rank of Well Concepts, this derives the appropriate color for them
    '''
    #Subset the WCs to only those which are on the creaming curve
    WC_rank = CalculateRankOfWCs([(a[0], a[1]) for a in WC_points],(plat_x,plat_y),radii)

    #Construct dataframe of all well concepts
    df_WC = pd.DataFrame({"Well Concept": WC_name,"Resource": WC_resource, "Rank":WC_rank})
    
    return df_WC['Rank'].map(lambda x: dct_colors[x])    

## Map Construction

https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9

In [66]:
lon_min = -60.64
lon_max = -60.1
lat_min = 9.85
lat_max = 10.5
n_concepts = 100

WC_name, WC_points, WC_resource = UpdateWellConceptData(n_concepts, lon_min, lon_max, lat_min, lat_max)

#Defaults
radiusinner=1
radiusouter=1.1


platlon = [-60.32]
platlat = [10.07]

#Any calculations based on the above defaults
WC_colors = GetWCColors(WC_points,platlon[0], platlat[0], [0,radiusinner,radiusouter])

inner_circx, inner_circy = GetCircumferenceXY([platlon[0], platlat[0]],radiusinner , 401)
outer_circx, outer_circy = GetCircumferenceXY([platlon[0], platlat[0]],radiusouter , 401)




In [67]:
import json
from bokeh.models.widgets import Slider

from bokeh.io import output_file, show, save

from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool, LabelSet,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
# Input GeoJSON source that contains features for plotting
from bokeh.models.markers import SquareCross


from bokeh.layouts import column, row, WidgetBox
from bokeh.models import Panel
from bokeh.models.widgets import Tabs

In [68]:
# Create figure object.
p = figure(title = 'Polygons', 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset",
            x_range=(lon_min, lon_max),
            y_range=(lat_min, lat_max),          
          )
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None



# TARGET LAYER #
# Data Source
ds_targ = GeoJSONDataSource(geojson = gdf_targPoly[['target_id', 'target_name', 'polygon']].to_json())

# Bokeh Layer
p_targ = p.patches('xs','ys', source = ds_targ,
                   fill_color = "lightskyblue",
                   line_color = "blue", 
                   line_width = 0.1, 
                   fill_alpha = 0.2)


#PLATFORM LAYER #
# Data Source 
ds_plat = ColumnDataSource({'x':platlon, 'y':platlat, 'name':["Platform"]})
# Bokeh Layer
p_plat = p.circle("x", "y", size=10, source=ds_plat,
         line_color="black", fill_alpha=0.8)
# Label layer
labels = LabelSet(x="x", y="y", text="name", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=ds_plat, text_align='center')
p.add_layout(labels)





#PLATFORM INNER STEP-OUT INNER #
#Data Source
source_step_inner = ColumnDataSource({'x':inner_circx, 'y':inner_circy})
#Bokeh layer
p_step_inner = p.patch('x','y', source = source_step_inner,
                   fill_color = None,
                   line_color = "#00CC00", 
                   line_width = 1, 
                   fill_alpha = 1)

#PLATFORM STEP-OUT OUTER #
source_step_outer = ColumnDataSource({'x':outer_circx, 'y':outer_circy})

p_step_outer = p.patch('x','y', source = source_step_outer,
                   fill_color = None,
                   line_color = "#FF8000", 
                   line_width = 1, 
                   fill_alpha = 1)





# WELL CONCEPTS
wcx = [a[0] for a in WC_points]
wcy = [a[1] for a in WC_points]
source_wc = ColumnDataSource({'x':wcx, 'y':wcy, 'name':WC_name, 'resource':WC_resource, 'hex': WC_colors})

p_wc = p.circle("x", "y", size=10, source=source_wc,
         color='hex', line_color="black", fill_alpha=0.8)

labels = LabelSet(x="x", y="y", text="name", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source_wc, text_align='center')
p.add_layout(labels)







# Create hover tool
p.add_tools(HoverTool(renderers = [p_wc],
                      tooltips = [('Well Concept','@name'),
                                ('Resource','@resource')]))

#output_file("p.html")


In [69]:
#Subset the WCs to only those which are on the creaming curve
WC_rank = CalculateRankOfWCs([(a[0], a[1]) for a in WC_points],(platlon[0],platlat[0]),[0,radiusinner, radiusouter])

#Construct dataframe of all well concepts
df_WC = pd.DataFrame({"Well Concept": WC_name,"Resource": WC_resource, "Rank":WC_rank})

#Reduce to df of well concepts on Creaming Curve (i.e. within the radii)
df_WC_cc = df_WC[df_WC['Rank'] != 0].copy()

#Map ranks to colors
df_WC_cc.loc[:,'Color'] = df_WC_cc['Rank'].map(lambda x: dct_colors[x])

df_WC_cc.sort_values(by="Resource", inplace=True, ascending=False)

b = figure(x_range=df_WC_cc['Well Concept'], plot_height=250, title="Creaming Curve")
b.vbar(x=df_WC_cc['Well Concept'], top=df_WC_cc['Resource'], color=df_WC_cc['Color'], width=0.9)

b.xgrid.grid_line_color = None
b.y_range.start = 0







In [70]:

output_file("p.html")

# Create a row, with control panel and plot
layout = row(p,b)


# Make a tab with the layout 
tab = Panel(child=layout, title = 'Geospatial PoC')
tabs = Tabs(tabs=[tab])





save(tabs)

'/Users/Pri.balachandran@ibm.com/Desktop/BP/20191106_PlatformWCAnalysis/p.html'

In [50]:
import sys
sys.path

['/Users/Pri.balachandran@ibm.com/opt/anaconda3/envs/py36-geographicplotting/lib/python36.zip',
 '/Users/Pri.balachandran@ibm.com/opt/anaconda3/envs/py36-geographicplotting/lib/python3.6',
 '/Users/Pri.balachandran@ibm.com/opt/anaconda3/envs/py36-geographicplotting/lib/python3.6/lib-dynload',
 '',
 '/Users/Pri.balachandran@ibm.com/opt/anaconda3/envs/py36-geographicplotting/lib/python3.6/site-packages',
 '/Users/Pri.balachandran@ibm.com/opt/anaconda3/envs/py36-geographicplotting/lib/python3.6/site-packages/IPython/extensions',
 '/Users/Pri.balachandran@ibm.com/.ipython']